# Recuerde hace una copia de este notebook en `File -> Save a copy in Drive`.

# Clustering

Utilizaremos un dataset de precios de automóviles: https://www.kaggle.com/code/goyalshalini93/car-price-prediction-linear-regression-rfe/data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("https://users.dcc.uchile.cl/~hsarmien/diplomado/dataset/CarPrice_Assignment.csv",
                         sep = ",",
                         encoding = "utf8")

In [ ]:
df.head(2)

In [ ]:
df.dtypes

In [ ]:
dfr = df[['price', 'fueltype', 'aspiration','carbody', 'drivewheel','wheelbase',
                  'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower',
                    'citympg', 'carlength','carwidth']]
dfr.head()

In [ ]:
def dummies(x,df):
    temp = pd.get_dummies(df[x], drop_first = True)
    df = pd.concat([df, temp], axis = 1)
    df.drop([x], axis = 1, inplace = True)
    return df

In [ ]:
dfr = dummies('fueltype',dfr)
dfr = dummies('aspiration',dfr)
dfr = dummies('carbody',dfr)
dfr = dummies('drivewheel',dfr)
dfr = dummies('enginetype',dfr)
dfr = dummies('cylindernumber',dfr)

In [ ]:
dfr.head(5)

In [ ]:
dfr.shape

In [ ]:
#from sklearn.preprocessing import LabelEncoder
#label_encoder = LabelEncoder()
#dfr.loc[:,"fueltype"] = label_encoder.fit_transform(dfr['fueltype'])

## K-Means

K-means es un método simple para particionar datos en distintos clusters, que intenta minimizar la distancia de cada punto de un cluster a su centroide.
Para ejemplificar, y conocer cómo usarlo en `scikit-learn`, haremos un ejemplo práctico donde se ven claramente 3 clusters:

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np



Ejecutamos k-means y le indicamos que queremos que divida los datos en 3 clusters:

In [ ]:
random_state = 20
kmeans = KMeans(n_clusters=3, n_init=20, max_iter=300, random_state=random_state)
kmeans.fit(dfr) # fit retorna a self
y_pred = kmeans.predict(dfr)

In [ ]:
dfr.head(3)

In [ ]:
y_pred

Lo que estamos haciendo es crear un objeto KMeans, que está configurado para asignar 3 clusters, y le damos un `random_state` para tener resultados replicables. `n_init` significa que vamos a correr el método `n_init` veces, donde cada vez que se inicie el método se generan centroides que parten de manera aleatoria, finalmente se queda con el resultado que tiene el error más bajo. `max_iter` nos dice el número máximo de iteraciones que hará el modelo en el caso de que no encuentre convergencia antes.

Luego pasamos los datos al modelo para que corra el algorimo de clustering. Finalmente le pedimos que nos de los clusters asignados a cada valor de entrada.

Con el objeto `kmeans` entrenado podemos preguntar por algunos de los resultados. Podemos revisar cuáles son los centroides finales del modelo con `cluster_centers_`, cuáles son las asignaciones por cada uno de los ejemplos con `labels_` (en este caso es lo mismo que `y_pred` que tenemos arriba), el error de los clusters con `inertia_` y cuántas iteraciones tomó encontrar este resultado con `n_iter_`.

In [ ]:
# aqui tenemos los centroides del modelo elegido
kmeans.cluster_centers_

También podemos obtenes medias de, por ejemplo, cuántos datos quedaron en cada cluster. No hay nada automático para obtener ese resultado, pero podemos usar `numpy` para contar los clusters.

In [ ]:
counts = np.bincount(y_pred)
print(counts)

# a veces tiene sentido filtrar los que tienen un valor mayor o igual a 0, ya que normalmente
# se asigna -1 cuando el dato es considerado ruido
# counts = np.bincount(y_pred[y_pred>=0])

Por otro lado, las asignaciones por cluster se pueden incorporar a una nueva columna en el dataset (representando el *cluster*). **Sin embargo, hay que tener cuidado al agregar esta columna al dataframe original, pues (erróneamente) podrían usarla a futuro en las variables para estimar otro modelo de clustering.**

In [ ]:
dfr.shape

In [ ]:
new_X = dfr.assign(cluster=kmeans.labels_)
new_X.head(10)

In [ ]:
new_X[new_X['cluster'] == 1].describe()

In [ ]:
new_X[new_X['cluster'] == 2].describe()

## Visualizando clusters con reducción de dimensionalidad

In [ ]:
from sklearn.decomposition import PCA
reduX = PCA(n_components=2, random_state=0).fit_transform(dfr)

#print(reduX)

In [ ]:
plt.scatter(reduX[:, 0], reduX[:, 1], c=kmeans.labels_)
plt.show()

In [ ]:
kmeans.inertia_

### Estimando la cantidad de clusters

En el ejemplo, creamos los datos nosotros, por lo que sabemos la cantidad de clusters que necesitamos. Sin embargo, esto no siempre es el caso, y como K-Means necesita este valor al momento de correr el algoritmo, no podemos dejarlo al azar.

Una forma de estimar el número de clusters es mediante la suma de la diferencia al cuadrado entre los puntos de cada cluster (SSE). En `scikit-learn` este dato se llama `inertia_`. Una tecnica para encontrar un número razonable de clusters a usar es el método del codo, donde calculamos el SSE para varios números de clusters y graficamos como varia el SSE, eligiendo el "mejor". Este concepto de "mejor" no es claro, pero la idea es elegir el último cluster antes de encontrarnos con el punto de _diminishing returns_, que sería cuando aumentar a más clusters nos da una mejora muy pequeña respecto a la que estamos considerando actualmente.

Veamos un ejemplo. Ejecutemos K-Means entre 1 y 15 clusters y grafiquemos cómo cambia el error a medida que aumentamos el número de clusters.

In [ ]:
sse = []

clusters = list(range(2, 16)) #range(1,41)
for k in clusters:
    kmeans = KMeans(n_clusters=k).fit(dfr)
    sse.append(kmeans.inertia_)
    #print("Kmeans silhouette para k = ", k, silhouette_score(dfr, kmeans.labels_))

    #plot_silhouette(dfr, kmeans)

plt.plot(clusters, sse, marker="o")
plt.title("Metodo del codo de 1 a 15 clusters")
plt.grid(True)
plt.show()

El gráfico nos muestra el error de K-Means usando diferentes números de clusters.
Acá se puede notar que un valor óptimo es 3 (mirar donde se forma el `codo` o el punto tras el cual el error decrece de manera menos significativa).
Si eligiéramos 4 o más, veríamos más particiones, pero posiblemente estaríamos separando clusters ya existentes en clusters más pequeños.
Ojo que este método es una heurística y no siempre el `codo` es claramente visible.

## Evaluación de clusters

Evaluar la calidad de nuestros clusters es algo no trivial. En 2 dimensiones podemos claramente encontrar separaciones de clusters visualmente, asumiendo que es tan simple como mirar un gráfico. El tema es que rara vez tendremos solo 2 dimensiones en nuestros datos, y cuando ya tenemos más de 3 dimensiones, no podemos graficarlo de una forma directa y tenemos que depender de técnicas de reducción de dimensionalidad. Se verá más adelante en el curso, pero las salidas de estas técnicas no son realmente interpretables.

En esta sección veremos algunas forma de evaluar clusters, sea visual o numéricamente.


### Matriz de similitud (proximidad)

Uno de los métodos vistos en clases para evaluar la calidad de los clusters es haciendo una matriz de similitud. Estas matrices nos permiten ver qué tan cerca están los puntos pertenecientes a un cluster entre sí, y simultaneamente ver qué tan lejos están los puntos de un cluster de los otros clusters.

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

def sim_matrix(features, labels):  #labels: a cual cluster pertenece cada dato [0,1,2,1,0]
    useful_labels = labels >= 0    #dbscan los datos outliers pertenecen al cluster -1

    # primero ordenamos los datos en base al cluster que pertencen
    indices = np.argsort(labels[useful_labels])
    sorted_features = features.iloc[indices]

    # calculamos las distancias entre todos los puntos
    d = euclidean_distances(sorted_features, sorted_features)
    return d

def plot_sim(data, model):
    fig, ax = plt.subplots()
    dist = sim_matrix(data, model.labels_)
    im = ax.imshow(dist, cmap="jet")
    fig.colorbar(im, ax=ax)

In [ ]:
kmeans1 = KMeans(n_clusters=3, random_state=random_state).fit(dfr)


In [ ]:
plot_sim(dfr, kmeans1)

### Silhouette

Presentaremos otra forma de evaluar clusters, esta vez de una manera no visual usando el coeficiente de Silhouette. Como se vio en clases, este coeficiente calcula para cada punto:

1) su distancia promedio al resto de los puntos en su mismo clases, digamos `a`. En ingles esto se llama `mean intra-cluster distance`.

2) su distancia promedio a todos los puntos del cluster mas cercano, digamos `b`. En ingles esto se llama `mean nearest-cluster distance`.
Entonces el coeficiente de Silhouette se calcula con la siguiente formula:
$$\frac{b - a}{max(a, b)}$$

Esta métrica esta en un rango entre -1 y 1, donde 1 significa que algo está bien asignado, -1 significa que algo está mal asignado porque hay otro cluster más similar, y 0 significa que hay solapamiento de clusters.

In [ ]:
from sklearn.metrics import silhouette_score

El coeficiente de Silhouette se calcula pasando el dataset y los labels asignados por el metodo de cluster:
```python
silhouette_score(<dataset>, <labels>)
```

Calculemos el Silhouette score de los modelos entrenados en la parte anterior.

In [ ]:
print("Kmeans silhouette", silhouette_score(dfr, kmeans1.labels_))


### Otra forma de ver Silhouette

En la parte anterior usamos el coeficiente para todos los datos. También podemos considerar el coeficiente para cada dato individual y graficar eso. También podemos usar esta técnica como una alternativa para encontrar el número de clusters que queremos usar.

In [ ]:
from sklearn.metrics import silhouette_samples

In [ ]:
def plot_silhouette(dataset, model):
    use_indices = model.labels_ >= 0
    use_labels = model.labels_[use_indices]
    use_data = dataset.iloc[use_indices]

    n_clusters = len(np.unique(use_labels))


    fig, ax1 = plt.subplots()

    silhouette_avg = silhouette_score(use_data, use_labels)
    print(f"The average silhouette_score for {model.__class__.__name__} is : {silhouette_avg}")
    sample_silhouette_values = silhouette_samples(use_data, use_labels)

    y_lower = 10
    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[use_labels == i]
        ith_cluster_silhouette_values.sort()
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                            0, ith_cluster_silhouette_values, alpha=0.7)
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10

    ax1.set_title(f"{model.__class__.__name__}")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    ax1.set_yticks([])



In [ ]:
plot_silhouette(dfr, kmeans1)

# DBSCAN

Algoritmo de clustering basado en densidad. Este método permite identificar clusters cuyos datos contienen mucho ruido, outliers y presentan una forma poco clara de separar en un plano. Pero tiene la debilidad de no funcionar bien cuando los clusters tienen densidades variables o tenemos una dimensionalidad muy alta.

DBSCAN está implementado en `scikit-learn` y necesita de los parametros `eps` y `min_samples`. `eps` corresponde a la distancia dentro de la cual se consideran 2 puntos vecinos, `min_samples` corresponde al `minpts` visto en clases, que es el número de vecinos que tiene que tener un punto para ser considerado un punto _core_.**bold text**

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import datasets

`eps` es el parametro más importante de DBSCAN, por lo que tenemos que elegirlo con cuidado. En este caso podemos ver que dice que hay 7 "clusters", de los cuales 6 son clusters reales y el resto es considerado como ruido.

In [ ]:
eps = 350   # 0.3
min_samples = 5

dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(dfr)

In [ ]:
from sklearn.decomposition import PCA
reduX = PCA(n_components=2, random_state=0).fit_transform(dfr)
plt.scatter(reduX[:, 0], reduX[:, 1], c=dbscan.labels_) # [1,0,-1,-1]
plt.title(f"DBSCAN: eps={eps}, min_samples={min_samples}")
plt.show()

In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
n_noise_ = list(dbscan.labels_).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

In [ ]:
print(dbscan.labels_)

In [ ]:
plot_sim(dfr, dbscan)

## Estimando eps con método de la rodilla

La idea de este procedimiento es calcular la distancia promedio de cada punto a sus `k` vecinos más cercanos los cuales son graficados en orden ascendente. El objetivo es determinar la _rodilla_, que corresponde al valor óptimo de `eps`.

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

nbrs = NearestNeighbors(n_neighbors=3).fit(dfr)
distances, indices = nbrs.kneighbors(dfr)

distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.axhline(y=700, color='r', linestyle='--') #Ajuste el valor para y
plt.plot(distances)

In [ ]:
X2, y2 = datasets.make_circles(n_samples=1500, factor=.5,noise=.05)

In [ ]:
plt.scatter(X2[:,0], X2[:,1])

Nosotros humanos vemos claramente que hay 2 grupos de datos. Sin embargo, K-Means no es capaz de separar esos 2 clusters, y los métodos aglomerativos necesitan de ayudas como mátrices de conectividad para lograrlo. Veamos como DBSCAN sí puede.

In [ ]:
eps = 0.2
min_samples = 5

dbscan_circles = DBSCAN(eps=eps, min_samples=min_samples).fit(X2)
plt.scatter(X2[:,0], X2[:,1], c=dbscan_circles.labels_)
plt.title(f"DBSCAN: eps={eps}, min_samples={min_samples}")
plt.show()

Aquí podemos ver otros ejemplos con otros datasets con que las otras técnicas de clustering tienen problemas.

In [ ]:
X3, y3 = datasets.make_moons(n_samples=1500, noise=.05)
plt.scatter(X3[:,0], X3[:,1])

In [ ]:
eps = 0.2
min_samples = 5

dbscan_moon = DBSCAN(eps=eps, min_samples=min_samples).fit(X3)
plt.scatter(X3[:,0], X3[:,1], c=dbscan_moon.labels_)
plt.title(f"DBSCAN: eps={eps}, min_samples={min_samples}")
plt.show()

In [ ]:

_filter_label = dbscan.labels_ >= 0
print("Dataset X DBSCAN\t", silhouette_score(dfr[_filter_label], dbscan.labels_[_filter_label]))

In [ ]:
plot_silhouette(dfr, dbscan)

## Clustering Jerárquico Aglomerativo (Hierarchical clustering)

Otra forma de hacer clustering es mediante Clustering Jerárquico Aglomerativo. En este método lo que hacemos es partir con que cada dato es un cluster independiente de los demás, y luego, mediante una matriz de distancias vamos uniendo datos, creando anidaciones de clusters. Continuamos hasta que quede solo 1 cluster muy grande.

Generalmente estos métodos se grafican como un dendrograma, y usan la distancia euclidiana para calcular las matrices de distancias. Dicho esto, se pueden usar otras métricas de distancia para calcular la matriz de afinidad pero en la mayoría de los casos usamos la distancia euclidiana.

Vamos a presentar 4 criterios para ir uniendo los clusters. Estos corresponden a `complete`, `average`, `single` y `ward`. Aquí una descripción rápida de los criterios:
* `complete`: considera la distancia máxima entre 2 clusters
* `average`: considera la distancia promedio entre 2 clusters
* `single`: considera la distancia mínima entre 2 clusters
* `ward`: minimiza la varianza entre los 2 clusters

Para trabajar con clustering jerárquico podemos usar `scikit-learn` o `scipy`. `scikit-learn` lamentablemente no tiene una forma directa de graficar los dendrogramas, pero `scipy` sí, así que presentaremos ambas por si alguna vez las necesitan.

En `scipy` existe todo un módulo dedicado a clustering jerárquico [scipy.cluster.hierarchy](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html). En particular aquí usaremos `linkage` para generar las uniones de cada dato y cluster, y `dendrogram` para graficar el árbol.

En `scikit-learn` tenemos [sklearn.cluster.AgglomerativeClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering) para computar los clusters y asignar los labels a cada dato.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

Aquí computamos los árboles usando cada uno de los criterios:

In [ ]:
complete = linkage(dfr, method="complete")
single = linkage(dfr, method="single")
average = linkage(dfr, method="average")
ward = linkage(dfr, method="ward")

Y ahora podemos graficar los árboles para ver como se distribuyen las ramas del árbol.

In [ ]:
dendrogram(complete)
plt.title("Linkage: Complete")
plt.show()

In [ ]:
dendrogram(single)
plt.title("Linkage: Single")
plt.show()

In [ ]:
dendrogram(average)
plt.title("Linkage: Average")
plt.show()

In [ ]:
dendrogram(ward)
plt.title("Linkage: Ward")
plt.show()

Visualmente podemos cortar el arbol en distintos puntos para ver cómo se distribuyen los datos en las ramas. Luego manualente decidir viendo el dendrograma cuál sería una buena distancia para cortar el árbol.

Por ejemplo, en el caso de `ward`, 60000 parece ser un buen número para cortar.

In [ ]:
dendrogram(ward)
plt.title("Linkage: Ward")
plt.axhline(y=60000, color='r', linestyle='--')
plt.show()

Hasta ahora hemos solo graficado los árboles, pero no hemos etiquetado los datos. Ahora podemos usar `scikit-learn` con `AgglomerativeClustering`. Aquí tenemos varias opciones.
* Si sabemos cuantos clusters queremos (viendo el dendrograma), agregamos el parametro `n_clusters` y lo dejamos en cuántos clusters queremos.
* Si sabemos a la distancia que queremos cortar (tambien viendo el dendrograma), entonces usamos el parametro `distance_threshold`.
* En el caso de que no usemos `linkage` podemos hacer correr el algoritmo y que genere todo el arbol (dejando `n_clusters=None` y `distance_threshold=0`), luego calcular la matriz de relaciones a mano, graficarla usando el dendrograma, decidir dónde cortar y volver a unos de los 2 puntos anteriores.

Corramos primero generando todo el árbol.

In [ ]:
ward_all = AgglomerativeClustering(n_clusters=None, linkage="ward", distance_threshold=0).fit(dfr)
print(ward_all.n_clusters_)

Como generamos el árbol entero tenemos tantos clusters como datos! Ahora, viendo el dendrograma anterior de `ward` decidimos que queremos 3 clusters. Entonces usamos lo siguiente:

In [ ]:
ward_3 = AgglomerativeClustering(n_clusters=3, linkage="ward").fit(dfr)
print(ward_3.n_clusters_)

Y también podemos obtener la etiquetas.

In [ ]:
ward_3.labels_

In [ ]:
from sklearn.decomposition import PCA
reduX = PCA(n_components=2, random_state=0).fit_transform(dfr)
plt.scatter(reduX[:, 0], reduX[:, 1], c=ward_3.labels_)
plt.show()